In [1]:
import pandas as pd
from collections import Counter
from glob import glob
import csv
import os.path as path

In [2]:
from pie_extended.cli.utils import get_tagger
from pie_extended.models.dum.imports import get_iterator_and_processor

In [81]:
path_name = 'txt/*.txt'

In [4]:
model_name = "dum"
tagger = get_tagger(model_name, batch_size=256, device="cpu", model_path=None)

2021-11-28 19:03:34,472 : Model /home/humanum/.local/lib/python3.8/site-packages/pie_extended/downloads/dum/lemma.tar was serialized with a previous version of `pie`. This might result in issues. Model commit is 050815e, whereas current `pie` commit is 9ce3e8f.
2021-11-28 19:03:34,996 : Model /home/humanum/.local/lib/python3.8/site-packages/pie_extended/downloads/dum/pos.tar was serialized with a previous version of `pie`. This might result in issues. Model commit is 050815e, whereas current `pie` commit is 9ce3e8f.


In [73]:
def dict_freq_tri_pos(list_token, doc_name):
    list_tri_pos = ['n_v_pron', 'pron_n_v', 'adp_pron_n', 'adp_art_n', 'v_pron_n', 'n_adv_v', 'n_adp_n', 'art_n_v', 'adv_v_pron', 'v_pron_v', 'n_pron_n', 'n_n_v', 'n_art_n', 'v_art_n', 'v_adv_v', 'n_v_v', 'n_conj_n', 'pron_v_pron', 'v_conj_pron', 'n_pron_v', 'art_n_n', 'pron_adv_v', 'v_adp_pron', 'n_adp_pron', 'n_conj_pron', 'pron_n_n', 'n_n_n', 'v_adp_n', 'n_v_adv', 'pron_n_pron', 'pron_v_v', 'v_pron_adv', 'art_n_adp', 'pron_n_adv', 'v_v_pron', 'n_v_conj', 'art_adj_n', 'adv_v_v', 'n_v_adp', 'pron_v_adp', 'adp_n_n', 'v_pron_pron', 'adp_n_v', 'adv_v_adv', 'conj_pron_v', 'pron_v_adv', 'adv_adv_v', 'n_n_pron', 'conj_pron_n', 'pron_pron_v']
    dict_result = dict.fromkeys(list_tri_pos)
    
    list_trigram = trigrammize(list_token)

    dict_temp = Counter(list_trigram)
    
    for key in dict_temp.keys():
        if key in dict_result.keys():
            dict_result[key] = dict_temp[key]/len(list_trigram)
    
    return dict_result

In [37]:
def trigrammize(list_token):
    """fonction qui prend en parametre une liste de tokens et retourne une liste de tri-grammes"""
    list_trigram = []
    for indice_token in range(len(list_token)-2):
        trigram = list_token[indice_token].split("(")[0]+'_'+list_token[indice_token+1].split("(")[0]+'_'+list_token[indice_token+2].split("(")[0]
        list_trigram.append(trigram)
    return list_trigram

In [72]:
def dict_freq_tri_char(list_tri_chars, doc_name):
    list_tri_chars_select = ['en ', 'er ', 'et ', 'n, ', 'en,', 't, ', ' de', 'den', 'an ', 'de ', ' he', 'ijn', ' ge', 'ver', 'een', ' ve', 'at ', 'oor', 'n d', 'aer', ' en', 'jn ', ' da', 'sch', 'eer', 'der', 'ck ', 'cht', 'ie ', ' va', ', D', ' ee', ' be', ' me', 'nde', ' we', 'aar', 'n s', 'ten', ' in', 't d', ' di', 'iet', ' mi', 'in ', 't g', 'ghe', 'te ', ' al', ' En']
    dict_result = dict.fromkeys(list_tri_chars_select)
    
    dict_temp = Counter(list_tri_chars)
    
    for key in dict_temp.keys():
        if key in dict_result.keys():
            dict_result[key] = dict_temp[key]/len(list_tri_chars)
    
    return dict_result

In [39]:
def tri_chars(chaine):
    list_three_chars = []
    for i in range(0, len(chaine)-2):
        list_three_chars.append(chaine[i]+chaine[i+1]+chaine[i+2])   
    return list_three_chars

In [71]:
def dict_freq_lemma(list_lemma, doc_name):
    list_lemma_select = ['een', 'de', 'ik', 'gij', 'hij', 'ne', 'in', 'mijn', 'van', 'het', 'uw', 'en', 'zijn', 'wij', 'zij', 'eten', 'niet', 'dat', 'te', 'met', 'op', 'die', 'haar', 'eer', 'voor', 'hebben', 'zo', 'al', 'als', 'aan', 'et', 'hoofd', 'na', 'tot', 'ook', 'door', 'hier', 'of', 'hoe', 'dan', 'geen', 'om', 'nu', 'wel', 'ons', 'haten', 'deze', 'wat', 'dit', 'bij']
    dict_result = dict.fromkeys(list_lemma_select)
    
    dict_temp = Counter(list_lemma)
    
    for key in dict_temp.keys():
        if key in dict_result.keys():
            dict_result[key] = dict_temp[key]/len(list_lemma)
    
    return dict_result

In [41]:
def nettoie_lemma(list_lemma, list_pos):
    pos_ok = ["pre", "adv", "det", "conj", "pron", "adp", "art"]
    list_nettoie = []
    if len(list_lemma) == len(list_pos):
        for i in range(0, len(list_lemma)-1):
            if list_pos[i].split("(")[0] in pos_ok:
                list_nettoie.append(list_lemma[i])
    return list_nettoie

In [178]:
def main(path_name, doc_name):
    with open(path_name, encoding="utf8") as file:
        lignes = file.readlines()

        for ligne in lignes:
            iterator, processor = get_iterator_and_processor()
            annotation = tagger.tag_str(ligne, iterator=iterator, processor=processor)
            list_three_chars = tri_chars(ligne)

    df_pie = pd.DataFrame(annotation)
    dict_result_pos = dict_freq_tri_pos(df_pie['pos'], doc_name)
         
    dict_result_char = dict_freq_tri_char(list_three_chars, doc_name)
    
    list_lemma = nettoie_lemma(df_pie['lemma'], df_pie['pos'])
    dict_result_lemma = dict_freq_lemma(list_lemma, doc_name)

    return dict_result_pos, dict_result_char, dict_result_lemma

In [82]:
df_main = moulinette(path_name)

In [83]:
df_main

,n_v_pron,pron_n_v,adp_pron_n,adp_art_n,v_pron_n,n_adv_v,n_adp_n,art_n_v,adv_v_pron,v_pron_v,...,geen,om,nu,wel,ons,haten,deze,wat,dit,bij
index,,,,,,,,,,,,,,,,,,,,,
13732,0.014815,0.007407,0.007407,0.022222,None,0.007407,0.014815,None,None,0.007407,...,0.014706,None,0.014706,None,None,None,0.014706,None,None,None
21533,0.003868,0.017408,0.007737,0.007737,0.023211,0.023211,None,0.005803,0.025145,0.025145,...,0.004032,0.004032,0.008065,0.016129,0.004032,0.008065,0.004032,0.004032,None,None
3542,0.017621,0.004405,None,None,0.004405,0.008811,0.004405,0.013216,None,0.008811,...,None,None,0.009009,None,None,0.009009,None,0.009009,None,None
186454,0.018018,None,0.018018,0.009009,None,None,0.036036,0.009009,0.018018,0.018018,...,0.017857,None,None,None,None,0.017857,0.035714,None,0.053571,0.017857
185051,0.015936,0.007968,0.011952,0.01992,0.007968,0.003984,0.003984,0.011952,0.015936,0.01992,...,0.007752,None,0.007752,None,0.007752,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117156,0.028226,0.016129,0.014113,0.014113,0.022177,0.012097,0.010081,0.006048,0.020161,0.022177,...,0.004065,0.020325,0.004065,0.02439,None,0.01626,0.00813,0.01626,0.00813,None
6213,0.018817,0.013441,0.016129,0.016129,0.010753,0.016129,0.005376,0.013441,0.008065,0.005376,...,0.005495,None,None,0.027473,0.010989,None,0.005495,0.016484,0.005495,0.005495
191412,0.011062,0.017699,0.017699,0.015487,0.022124,0.00885,0.004425,0.002212,0.006637,0.004425,...,0.009091,None,0.009091,None,None,0.004545,None,None,None,None


In [84]:
df_main.to_csv(r'features.csv', index = True)

In [169]:
def moulinette(path_name):
    i = 0
    dict_results = {}

    df_main = pd.DataFrame()
    
    for doc in glob(path_name):
        doc_name = path.splitext(path.basename(doc))[0]
        print(doc_name)
        dict_result_pos, dict_result_char, dict_result_lemma = main(doc, doc_name)
        
        dict_results.update(dict_result_pos)
        dict_results.update(dict_result_char)
        dict_results.update(dict_result_lemma)
        
        dict_results["index"] = doc_name
        
        
        df_temp = pd.DataFrame(dict_results, index=[0])
        
        df_main = df_main.append(df_temp, ignore_index = True)
        
        i+=1
        
        print("PROGRESSION ", round((i/22000)*100,3),'% COMPLETED\n')
        
        
    df_main.set_index("index", inplace = True)
    
    return df_main

In [172]:
path_name = 'txt_train/*.txt'
#path_name = 'test/*.txt'

In [179]:
df_train = moulinette(path_name)

33953
PROGRESSION  0.005 % COMPLETED

191753
PROGRESSION  0.009 % COMPLETED

153521
PROGRESSION  0.014 % COMPLETED

22403
PROGRESSION  0.018 % COMPLETED

185146
PROGRESSION  0.023 % COMPLETED

35335
PROGRESSION  0.027 % COMPLETED

188983
PROGRESSION  0.032 % COMPLETED

148209
PROGRESSION  0.036 % COMPLETED

20876
PROGRESSION  0.041 % COMPLETED

166979
PROGRESSION  0.045 % COMPLETED

118968
PROGRESSION  0.05 % COMPLETED

2041
PROGRESSION  0.055 % COMPLETED

182067
PROGRESSION  0.059 % COMPLETED

158390
PROGRESSION  0.064 % COMPLETED

189167
PROGRESSION  0.068 % COMPLETED

166066
PROGRESSION  0.073 % COMPLETED

111364
PROGRESSION  0.077 % COMPLETED

185135
PROGRESSION  0.082 % COMPLETED

115395
PROGRESSION  0.086 % COMPLETED

100153
PROGRESSION  0.091 % COMPLETED

158610
PROGRESSION  0.095 % COMPLETED

22439
PROGRESSION  0.1 % COMPLETED

157178
PROGRESSION  0.105 % COMPLETED

20020
PROGRESSION  0.109 % COMPLETED

176145
PROGRESSION  0.114 % COMPLETED

11388
PROGRESSION  0.118 % COMPLETED

PROGRESSION  0.977 % COMPLETED

191506
PROGRESSION  0.982 % COMPLETED

188646
PROGRESSION  0.986 % COMPLETED

175208
PROGRESSION  0.991 % COMPLETED

179643
PROGRESSION  0.995 % COMPLETED

160285
PROGRESSION  1.0 % COMPLETED

184480
PROGRESSION  1.005 % COMPLETED

117533
PROGRESSION  1.009 % COMPLETED

134391
PROGRESSION  1.014 % COMPLETED

123218
PROGRESSION  1.018 % COMPLETED

121093
PROGRESSION  1.023 % COMPLETED

100032
PROGRESSION  1.027 % COMPLETED

1875
PROGRESSION  1.032 % COMPLETED

149135
PROGRESSION  1.036 % COMPLETED

11054
PROGRESSION  1.041 % COMPLETED

13374
PROGRESSION  1.045 % COMPLETED

32525
PROGRESSION  1.05 % COMPLETED

19347
PROGRESSION  1.055 % COMPLETED

21234
PROGRESSION  1.059 % COMPLETED

123727
PROGRESSION  1.064 % COMPLETED

3682
PROGRESSION  1.068 % COMPLETED

18354
PROGRESSION  1.073 % COMPLETED

21129
PROGRESSION  1.077 % COMPLETED

123290
PROGRESSION  1.082 % COMPLETED

120358
PROGRESSION  1.086 % COMPLETED

20714
PROGRESSION  1.091 % COMPLETED

117779
P

PROGRESSION  1.955 % COMPLETED

4155
PROGRESSION  1.959 % COMPLETED

8869
PROGRESSION  1.964 % COMPLETED

173908
PROGRESSION  1.968 % COMPLETED

177166
PROGRESSION  1.973 % COMPLETED

36662
PROGRESSION  1.977 % COMPLETED

3822
PROGRESSION  1.982 % COMPLETED

117718
PROGRESSION  1.986 % COMPLETED

20699
PROGRESSION  1.991 % COMPLETED

10722
PROGRESSION  1.995 % COMPLETED

13304
PROGRESSION  2.0 % COMPLETED

61167
PROGRESSION  2.005 % COMPLETED

157045
PROGRESSION  2.009 % COMPLETED

3433
PROGRESSION  2.014 % COMPLETED

176804
PROGRESSION  2.018 % COMPLETED

104002
PROGRESSION  2.023 % COMPLETED

190711
PROGRESSION  2.027 % COMPLETED

173316
PROGRESSION  2.032 % COMPLETED

157904
PROGRESSION  2.036 % COMPLETED

13125
PROGRESSION  2.041 % COMPLETED

115259
PROGRESSION  2.045 % COMPLETED

120349
PROGRESSION  2.05 % COMPLETED

2540
PROGRESSION  2.055 % COMPLETED

192558
PROGRESSION  2.059 % COMPLETED

35956
PROGRESSION  2.064 % COMPLETED

172877
PROGRESSION  2.068 % COMPLETED

123316
PROGRE

PROGRESSION  2.927 % COMPLETED

180744
PROGRESSION  2.932 % COMPLETED

11904
PROGRESSION  2.936 % COMPLETED

36079
PROGRESSION  2.941 % COMPLETED

9163
PROGRESSION  2.945 % COMPLETED

166983
PROGRESSION  2.95 % COMPLETED

1628
PROGRESSION  2.955 % COMPLETED

186609
PROGRESSION  2.959 % COMPLETED

180328
PROGRESSION  2.964 % COMPLETED

100261
PROGRESSION  2.968 % COMPLETED

11060
PROGRESSION  2.973 % COMPLETED

183271
PROGRESSION  2.977 % COMPLETED

8988
PROGRESSION  2.982 % COMPLETED

185025
PROGRESSION  2.986 % COMPLETED

191520
PROGRESSION  2.991 % COMPLETED

188460
PROGRESSION  2.995 % COMPLETED

35957
PROGRESSION  3.0 % COMPLETED

8953
PROGRESSION  3.005 % COMPLETED

117668
PROGRESSION  3.009 % COMPLETED

192778
PROGRESSION  3.014 % COMPLETED

180716
PROGRESSION  3.018 % COMPLETED

163709
PROGRESSION  3.023 % COMPLETED

38794
PROGRESSION  3.027 % COMPLETED

156530
PROGRESSION  3.032 % COMPLETED

187628
PROGRESSION  3.036 % COMPLETED

20311
PROGRESSION  3.041 % COMPLETED

160069
PRO

PROGRESSION  3.909 % COMPLETED

162083
PROGRESSION  3.914 % COMPLETED

9558
PROGRESSION  3.918 % COMPLETED

181946
PROGRESSION  3.923 % COMPLETED

160449
PROGRESSION  3.927 % COMPLETED

150267
PROGRESSION  3.932 % COMPLETED

192796
PROGRESSION  3.936 % COMPLETED

123640
PROGRESSION  3.941 % COMPLETED

20284
PROGRESSION  3.945 % COMPLETED

164630
PROGRESSION  3.95 % COMPLETED

161185
PROGRESSION  3.955 % COMPLETED

11818
PROGRESSION  3.959 % COMPLETED

122915
PROGRESSION  3.964 % COMPLETED

40784
PROGRESSION  3.968 % COMPLETED

115648
PROGRESSION  3.973 % COMPLETED

174659
PROGRESSION  3.977 % COMPLETED

136000
PROGRESSION  3.982 % COMPLETED

187947
PROGRESSION  3.986 % COMPLETED

193293
PROGRESSION  3.991 % COMPLETED

14166
PROGRESSION  3.995 % COMPLETED

186301
PROGRESSION  4.0 % COMPLETED

14142
PROGRESSION  4.005 % COMPLETED

152991
PROGRESSION  4.009 % COMPLETED

34047
PROGRESSION  4.014 % COMPLETED

10592
PROGRESSION  4.018 % COMPLETED

11386
PROGRESSION  4.023 % COMPLETED

117700

...

PROGRESSION  96.3 % COMPLETED

25660
PROGRESSION  96.305 % COMPLETED

100210
PROGRESSION  96.309 % COMPLETED

34258
PROGRESSION  96.314 % COMPLETED

182413
PROGRESSION  96.318 % COMPLETED

14661
PROGRESSION  96.323 % COMPLETED

9848
PROGRESSION  96.327 % COMPLETED

166088
PROGRESSION  96.332 % COMPLETED

185128
PROGRESSION  96.336 % COMPLETED

137261
PROGRESSION  96.341 % COMPLETED

115711
PROGRESSION  96.345 % COMPLETED

8788
PROGRESSION  96.35 % COMPLETED

188436
PROGRESSION  96.355 % COMPLETED

23560
PROGRESSION  96.359 % COMPLETED

156202
PROGRESSION  96.364 % COMPLETED

193347
PROGRESSION  96.368 % COMPLETED

185361
PROGRESSION  96.373 % COMPLETED

112140
PROGRESSION  96.377 % COMPLETED

177618
PROGRESSION  96.382 % COMPLETED

34050
PROGRESSION  96.386 % COMPLETED

148214
PROGRESSION  96.391 % COMPLETED

118612
PROGRESSION  96.395 % COMPLETED

182542
PROGRESSION  96.4 % COMPLETED

14540
PROGRESSION  96.405 % COMPLETED

155985
PROGRESSION  96.409 % COMPLETED

8013
PROGRESSION  96.4

PROGRESSION  97.255 % COMPLETED

61992
PROGRESSION  97.259 % COMPLETED

187218
PROGRESSION  97.264 % COMPLETED

34619
PROGRESSION  97.268 % COMPLETED

4192
PROGRESSION  97.273 % COMPLETED

32711
PROGRESSION  97.277 % COMPLETED

2503
PROGRESSION  97.282 % COMPLETED

123621
PROGRESSION  97.286 % COMPLETED

18863
PROGRESSION  97.291 % COMPLETED

15413
PROGRESSION  97.295 % COMPLETED

7012
PROGRESSION  97.3 % COMPLETED

112060
PROGRESSION  97.305 % COMPLETED

24588
PROGRESSION  97.309 % COMPLETED

120603
PROGRESSION  97.314 % COMPLETED

8969
PROGRESSION  97.318 % COMPLETED

10748
PROGRESSION  97.323 % COMPLETED

20612
PROGRESSION  97.327 % COMPLETED

185344
PROGRESSION  97.332 % COMPLETED

148024
PROGRESSION  97.336 % COMPLETED

20776
PROGRESSION  97.341 % COMPLETED

21565
PROGRESSION  97.345 % COMPLETED

158287
PROGRESSION  97.35 % COMPLETED

27067
PROGRESSION  97.355 % COMPLETED

191338
PROGRESSION  97.359 % COMPLETED

118731
PROGRESSION  97.364 % COMPLETED

186547
PROGRESSION  97.368 % 

PROGRESSION  98.205 % COMPLETED

6818
PROGRESSION  98.209 % COMPLETED

6313
PROGRESSION  98.214 % COMPLETED

178630
PROGRESSION  98.218 % COMPLETED

116933
PROGRESSION  98.223 % COMPLETED

184513
PROGRESSION  98.227 % COMPLETED

136136
PROGRESSION  98.232 % COMPLETED

130189
PROGRESSION  98.236 % COMPLETED

26515
PROGRESSION  98.241 % COMPLETED

34255
PROGRESSION  98.245 % COMPLETED

21402
PROGRESSION  98.25 % COMPLETED

26456
PROGRESSION  98.255 % COMPLETED

117508
PROGRESSION  98.259 % COMPLETED

120820
PROGRESSION  98.264 % COMPLETED

20011
PROGRESSION  98.268 % COMPLETED

20663
PROGRESSION  98.273 % COMPLETED

15997
PROGRESSION  98.277 % COMPLETED

36010
PROGRESSION  98.282 % COMPLETED

20100
PROGRESSION  98.286 % COMPLETED

8886
PROGRESSION  98.291 % COMPLETED

188906
PROGRESSION  98.295 % COMPLETED

19251
PROGRESSION  98.3 % COMPLETED

32573
PROGRESSION  98.305 % COMPLETED

176886
PROGRESSION  98.309 % COMPLETED

1835
PROGRESSION  98.314 % COMPLETED

20803
PROGRESSION  98.318 % C

PROGRESSION  99.155 % COMPLETED

1871
PROGRESSION  99.159 % COMPLETED

120308
PROGRESSION  99.164 % COMPLETED

51683
PROGRESSION  99.168 % COMPLETED

3713
PROGRESSION  99.173 % COMPLETED

7225
PROGRESSION  99.177 % COMPLETED

14598
PROGRESSION  99.182 % COMPLETED

116996
PROGRESSION  99.186 % COMPLETED

2495
PROGRESSION  99.191 % COMPLETED

184178
PROGRESSION  99.195 % COMPLETED

190377
PROGRESSION  99.2 % COMPLETED

4754
PROGRESSION  99.205 % COMPLETED

117722
PROGRESSION  99.209 % COMPLETED

184806
PROGRESSION  99.214 % COMPLETED

36223
PROGRESSION  99.218 % COMPLETED

34466
PROGRESSION  99.223 % COMPLETED

116612
PROGRESSION  99.227 % COMPLETED

20793
PROGRESSION  99.232 % COMPLETED

111899
PROGRESSION  99.236 % COMPLETED

11909
PROGRESSION  99.241 % COMPLETED

11877
PROGRESSION  99.245 % COMPLETED

179137
PROGRESSION  99.25 % COMPLETED

4367
PROGRESSION  99.255 % COMPLETED

10746
PROGRESSION  99.259 % COMPLETED

22276
PROGRESSION  99.264 % COMPLETED

185407
PROGRESSION  99.268 % CO

PROGRESSION  100.105 % COMPLETED

34698
PROGRESSION  100.109 % COMPLETED

178529
PROGRESSION  100.114 % COMPLETED

176279
PROGRESSION  100.118 % COMPLETED

163011
PROGRESSION  100.123 % COMPLETED

35406
PROGRESSION  100.127 % COMPLETED

20555
PROGRESSION  100.132 % COMPLETED

182426
PROGRESSION  100.136 % COMPLETED

20152
PROGRESSION  100.141 % COMPLETED

187769
PROGRESSION  100.145 % COMPLETED

3813
PROGRESSION  100.15 % COMPLETED

20395
PROGRESSION  100.155 % COMPLETED

192604
PROGRESSION  100.159 % COMPLETED

160288
PROGRESSION  100.164 % COMPLETED

129632
PROGRESSION  100.168 % COMPLETED

6949
PROGRESSION  100.173 % COMPLETED

123120
PROGRESSION  100.177 % COMPLETED

22227
PROGRESSION  100.182 % COMPLETED

2112
PROGRESSION  100.186 % COMPLETED

22162
PROGRESSION  100.191 % COMPLETED

130720
PROGRESSION  100.195 % COMPLETED

5064
PROGRESSION  100.2 % COMPLETED

9693
PROGRESSION  100.205 % COMPLETED

26396
PROGRESSION  100.209 % COMPLETED

2760
PROGRESSION  100.214 % COMPLETED

3800


In [180]:
df_train

,n_v_pron,pron_n_v,adp_pron_n,adp_art_n,v_pron_n,n_adv_v,n_adp_n,art_n_v,adv_v_pron,v_pron_v,...,geen,om,nu,wel,ons,haten,deze,wat,dit,bij
index,,,,,,,,,,,,,,,,,,,,,
33953,0.016949,0.008475,0.012712,0.021186,0.004237,0.012712,0.008475,0.008475,0.012712,None,...,0.008547,None,None,0.008547,None,None,0.025641,0.008547,None,0.008547
191753,0.014388,0.017986,0.028777,0.007194,0.007194,0.017986,0.003597,0.003597,0.017986,0.014388,...,None,0.006579,0.006579,0.006579,None,None,None,None,None,None
153521,None,None,0.016393,None,None,0.016393,0.032787,None,None,0.016393,...,None,0.037037,None,None,None,0.037037,None,None,0.074074,None
22403,0.041379,0.013793,0.013793,0.034483,0.027586,0.006897,None,0.006897,0.006897,0.006897,...,0.013889,None,None,None,0.013889,None,None,None,0.027778,None
185146,0.004975,None,0.007463,0.00995,0.007463,0.002488,0.004975,0.004975,0.002488,0.004975,...,None,None,0.005051,0.005051,None,None,0.005051,0.010101,None,0.010101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120328,0.027273,0.027273,0.009091,0.004545,0.013636,0.004545,None,0.004545,0.009091,0.022727,...,0.033613,0.008403,0.008403,0.008403,None,0.008403,None,None,None,0.008403
115335,0.011628,0.011628,0.011628,None,0.011628,None,0.034884,None,None,None,...,None,None,0.027027,None,None,0.027027,None,None,None,None
36610,0.008772,0.02193,0.02193,0.013158,0.008772,None,0.008772,0.013158,0.008772,0.004386,...,0.015504,0.015504,None,0.007752,None,0.007752,None,None,0.007752,None


In [181]:
df_train.to_csv(r'features_train.csv', index = True)